### 랭체인을 표현하는 언어 LCEL
- 여기까지는 colab에서 실습가능
- 이후부터는 로컬환경에서 실습가능

#### 환경설정

In [1]:
# #필수 라이브러리 설치
!pip install --upgrade --quiet langchain openai langchain-core langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 1.7 MB/s eta 0:00:00


In [7]:
# 01 colab용 비밀 키 불러오기
from google.colab import userdata
import os
from openai import OpenAI # openai-python SDK

# 02 OpenAI API키 설정
api_key = userdata.get("modulabsKey") # colab 사용자 환경변수 탭에서 등록한 키
os.environ["OPEN_API_KEY"] = api_key
client = OpenAI(api_key = api_key)

# 03 버전 확인(선택)
import openai
import langchain
print("LangChain ver.",langchain.__version__)
print("OpenAI SDK ver.", openai.__version__)

LangChain ver. 0.3.26
OpenAI SDK ver. 1.91.0


#### 01 LCEL로 기본 체인 구성

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os

# 프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")

# api_key 명시적으로 불러오기()
api_key = userdata.get("modulabsKey")
os.environ["OPENAI_API_KEY"] = api_key

# LLM 호출
model = ChatOpenAI(model = "gpt-4o-mini")

# 출력 파서 설정
output_parser = StrOutputParser()

# LCEL로 프롬프트 템플릿 - LLM-출력 파서 연결하기
chain = prompt | model | output_parser

# invoke 함수로 chain 실행하기
chain.invoke({"topic" : "ice cream"})

'Why did the ice cream cone break up with the sundae?  \nBecause it found someone cooler!'

#### 02 Streaming 기능 추가

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# chain 선언
model = ChatOpenAI(model = "gpt-4o-mini")

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = prompt | model

# chain의 stream() 함수를 통해 스트리밍 기능 추가
for s in chain.stream({"topic": "bears"}):
  print(s.content, end = "", flush = True)

Why do bears have hairy coats?

Because they look silly in sweaters!

#### 03 여러 개 API요청받는 batch()
- 랭체인은 RAG 외에도 다양한 서비스에 접목하거나, 자연어 처리를 위해 사용될 수 있음
- 여러 입력 값, 출력 값 한 번에 얻고 싶을 때 LCEL의 batch()를 활용하면 쉽게 구현할 수 있음

##### 01) 5개 문장 번역 batch() 수행

In [15]:
%%time
model = ChatOpenAI(model="gpt-4o-mini")
prompt = ChatPromptTemplate.from_template("다음 한글 문장을 프랑스어로 번역해줘 {sentence}")
chain = prompt | model

chain.batch([
    {"sentence": "그녀는 매일 아침 책을 읽습니다."},
    {"sentence": "오늘 날씨가 참 좋네요."},
    {"sentence": "저녁에 친구들과 영화를 볼 거예요."},
    {"sentence": "그 학생은 매우 성실하게 공부합니다."},
    {"sentence": "커피 한 잔이 지금 딱 필요해요."}

])

CPU times: user 76.8 ms, sys: 8.22 ms, total: 85 ms
Wall time: 1.17 s


[AIMessage(content='그녀는 매일 아침 책을 읽습니다.는 프랑스어로 "Elle lit un livre tous les matins."입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 33, 'total_tokens': 63, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BmWGS6tK7ls9T8nVA2gYaOkShJqgX', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--1c67e30d-29f1-4372-a653-5f38c93876e9-0', usage_metadata={'input_tokens': 33, 'output_tokens': 30, 'total_tokens': 63, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 AIMessage(content='오늘 날씨가 참 좋네요. 를 프랑스어로 번역하면 "Aujourd\'hui, le temps est vraiment agréable."입니다.', additional_kwargs={'refusal': N

##### 02) 1개 문장 번역 invoke() 수행

In [16]:
%%time
model = ChatOpenAI(model = 'gpt-4o-mini')
prompt = ChatPromptTemplate.from_template("다음 한글 문장을 프랑스어로 번역해줘 {sentence}")
chain = prompt | model

chain.invoke({"sentence": "그녀는 매일 아침 책을 읽습니다."})

CPU times: user 21.8 ms, sys: 2.04 ms, total: 23.8 ms
Wall time: 841 ms


AIMessage(content='그녀는 매일 아침 책을 읽습니다.를 프랑스어로 번역하면 다음과 같습니다.\n\n"Elle lit un livre tous les matins."', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 33, 'total_tokens': 68, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BmWKc4CmEBcfpmMirhFkVzPPhH9Qu', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--7266ef2c-6c8b-428f-9117-fd7876ee3f92-0', usage_metadata={'input_tokens': 33, 'output_tokens': 35, 'total_tokens': 68, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})